***
<h2> <u> First steps with tensorflow </u></h2>

* In this part of the practical we will learn about `tensorflow` (tf) and `numpy` (np) and how the two packages can be used together.
* In particular, we learn about the difference between `tf.placeholder`, `tf.Variable` and `np.array`.
* In a nutshell the three can be defined as follows:
  * `np.array`: Holds arrays of data (usually numbers) of a certain type (often float32). 
  * `tf.placeholder`: Is a node in a computational graph and does not have a fixed value. Rather they are like symbols in a maths equation until you plug in a value. Expressions of multiple placeholders (i.e. computational graphs) can only be evaluated in a `tf.Session`. 
  * `tf.Variable`: Variables are a hybrid of the two above. They are like placeholders because they can be part of expressions with placeholders, but they also have a specific value. This value can be changed over time, this is why `tf.Variables` are usually used for parameters we want to optimise, such as network weights. 
 
*** 
Let's start by importing the two packages: 

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import os

<h2> <u>Mount Google drive folder</u> </h2>

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/ML_workshop

In [ ]:
ls

***
<h3> <u> Placeholders </u></h3>

Let's create two placeholders `x_pl` and `y_pl` and formulate an expression combining the two. 

`tf.placeholder` takes the datatype as mandatory input. Optionally, you can also specify it's `shape` and give it a `name`. A known shape can be useful for more complicated expressions and will reduce bugs. Similarly, giving it a name makes debugging easier.

Try defining the following function `f_pl`: 

$ f(x,y) = x^2 + y^2 $



In [ ]:
x_pl = tf.placeholder(tf.float32, name='x')
y_pl = tf.placeholder(tf.float32, name='y')

### TO DO 1 ####
# f_pl = ...
#######################

print(x_pl)
print(y_pl)
print(f_pl)

***
**What is a tensor?:** Tensors are [loosely speaking](https://physics.stackexchange.com/questions/20437/are-matrices-and-second-rank-tensors-the-same-thing) generalisations of matrices (2D) to other dimensions (ND). When working with images in deep learning one often works with 4D arrays of the form [batch_size, image_size_x, image_size_y, n_color_channels]. In tensorflow all Variables and placeholders are called tensors. 

***
As mentioned earlier, this expression can only be evaluated inside a `tf.Session`. Sessions have a function called `run`, 
which we can use to evaluate expressions. In order to evaluate `f_pl`, we need to define values for `x_pl` and `y_pl`. 
This is done using the `feed_dict` argument, as in the example below.

Note that the `feed_dict` needs to be a [python dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries), which has the synatx: `{key1: value1, key2: value2, ...}`.

In [ ]:
with tf.Session() as sess:
    
    f_evaluated = sess.run(f_pl, feed_dict={x_pl: 1, y_pl: 2})

print('The value of f evaluated at the given values is: %f' % f_evaluated)

Take good note of the above syntax. We will be using `sess.run(...)` with values fed through a `feed_dict` a lot throughout this tutorial. 

We can do many things with placeholders that one can also do with algebraic expressions. As an example the function `tf.gradients(a,b)` can be used to calculate the derivative of `a` w.r.t. `b`. In the cell below, use this `tf.gradients` function to implement an expression for: $ \frac{\partial f}{\partial x} $

In [ ]:
# compute gradients of f_pl wrt x_pl
df_dx_pl = tf.gradients(f_pl, x_pl)

with tf.Session() as sess:  # using the with statement we the Session is automatically closed at the end of
                            # expressions which are inside the with statement.     
    df_dx_evaluated = sess.run(df_dx_pl, feed_dict={x_pl: 1, y_pl: 2})
    
# The result returned in df_dx_evaluated is actually an array, so we need to extract the first, and only value using [0]
print('The gradient df_dx evaluated at the given values is: %f' % df_dx_evaluated[0])

<h3> <u>Numpy arrays</u> </h3>

* You are likely to have encountered numpy arrays before.
* Here we create two sample arrays `a` and `b` and then show how to use them in a tensorflow expression. 

In [ ]:
A = np.array([[1,2],[3,4]])
B = 4.2*np.ones((2,2))

print('The two numpy arrays have values:')
print(A)
print(B)

Fortunately, working with `np.arrays` is no different from working with scalars. Since we didn't specify a shape for `x_pl` and `y_pl` above, we can now simply use them again assuming they are 2x2 arrays. 

`tf.matmul(x,y)` is the tensorflow function used for matrix multiplication. Below, implement expressions for 
 - $g(X,Y) = XY$, and
 - $\tfrac{\partial g}{\partial X}$  
 
Note that the usual multiplication symbol (`*`) will lead to an element wise matrix multiplication instead. For the proper matrix multiplication `tf.matmul` needs to be used. 

In [ ]:
### TO DO 2 ####
# g_pl = ...
# dg_dx_pl = ...
###############################

with tf.Session() as sess:
    
    # Note that sess.run can also be used to evaluate multiple expressions at once
    g_evaluated, dg_dx_evaluated = sess.run([g_pl, dg_dx_pl], feed_dict={x_pl: A, y_pl: B})
    
print('Function g(.) evaluated:')
print(g_evaluated)

print('Function dg/dx evaluated:')
print(dg_dx_evaluated[0])

### Shared variables 

Shared variables are symbolic variables that have a persistent value, as well. They are a sort of hybrid between numpy arrays and placeholders. However, Variables need to be initialised *within a session* before they can be used. To this end, first, a tensorflow init operation has to be defined, and then executed with `sess.run(...)`. 

Note that when evaluating a Variable, there is no need to feed it a value using `feed_dict` because it already has a value. In the example below use the `sess.run` function to run the `init_op`.


In [ ]:
i_var = tf.Variable(0.0)
init_op = tf.global_variables_initializer()  # operation to initialise all tf.Variables in the scope

with tf.Session() as sess:
    
    sess.run(init_op)  # init_op needs to be executed once before i_var can be used. 
    
    i_evaluated = sess.run(i_var)  # No need for a feed_dict here, as i_var does not depend on the value of any placeholder
    
print('The value of i_var is: %f' % i_evaluated)

Nothing too surprising here.

Because Variables have persistent values (within the context of a tf.Session), they are usually used for the parameters that are optimised in the learning process (e.g. the network weights and biases). 

We can for instance, define an operation that increments the value of `i_var` each time you run it. 

In [ ]:
increment_op = tf.assign(i_var, i_var + 1)

sess = tf.Session()  # Here we don't use the with statement, because we want the Session to stay active

sess.run(init_op)  # Note that we need to run this again because the session from above is closed again.

**Evaluate the following cell multiple times.**

In [ ]:
# run the increment operation.
sess.run(increment_op)    
i_evaluated = sess.run(i_var)
print('The value of i_var is: %f' % i_evaluated)

Lastly, Variables can be combined with placeholders in expressions. Try writing an expression for $ x^i $ (for a scalar x) using the Variable `i_var` defined above and evaluate it in a session. And evaluate it for a range of x from 0 to 4. 

In [ ]:
### TO DO 3 ####
power_pl = x_pl**i_var
######################

for x in range(5):
    
    ### IMPLEMENT THIS ###
    # power_evaluated = ...
    ######################
    
    print(' - %.2f^i = %.2f' % (x, power_evaluated))

Lastly, because a `tf.Session()` may own resources, such as the variable we defined, it is important to release these resources when they are no longer required. Since we didn't use the `with` statement here, we need to invoke the `close()` method to do this.

In [ ]:
sess.close()